In [1]:
%cd ..

/kaggle/working


In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="../cand_supervised/prob_matrix_filter"):
    cfg = compose(config_name="config.yaml", overrides=["exp=two002"])
    print(OmegaConf.to_yaml(cfg))

debug: false
seed: 7
dir:
  data_dir: /kaggle/working/input/atmaCup16_Dataset
  output_dir: /kaggle/working/output
  exp_dir: /kaggle/working/output/exp
  cand_unsupervised_dir: /kaggle/working/output/cand_unsupervised
  cand_supervised_dir: /kaggle/working/output/cand_supervised
  datasets_dir: /kaggle/working/output/datasets
exp:
  num_candidate: 100
  k:
  - 1
  - 5
  - 10
  - 50
  - 100
  range_transitions:
  - -1
  - 1
  transition_times: 2
  self_loop_prob: 0.5



In [3]:
import logging
import os
import pickle
import sys
import time
from pathlib import Path

import hydra
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import scipy.sparse as sparse
from hydra.core.hydra_config import HydraConfig
from omegaconf import DictConfig, OmegaConf
from scipy.sparse import csr_matrix, eye
from sklearn.preprocessing import normalize
from tqdm.auto import tqdm

import utils
import wandb
from utils.load import load_label_data, load_log_data, load_session_data, load_yad_data
from utils.logger import get_logger
from utils.metrics import calculate_metrics

In [4]:
with utils.timer("load data"):
    train_log_df = load_log_data(Path(cfg.dir.data_dir), "train")
    train_label_df = load_label_data(Path(cfg.dir.data_dir))
    test_log_df = load_log_data(Path(cfg.dir.data_dir), "test")

    # 最後にlabelが来るとしたときのseq_noを計算しておく
    last_seq_df = train_log_df.group_by("session_id").agg(pl.col("seq_no").max() + 1)
    # labelを付与
    label_seq_df = last_seq_df.join(train_label_df, on="session_id")
    all_log_df = pl.concat([train_log_df, label_seq_df, test_log_df]).sort(
        ["session_id", "seq_no"]
    )

[load data] done in 0.1 s
